In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# --- 1. Inicialização do Spark ---
spark = SparkSession.builder \
    .appName("CargaGold") \
    .getOrCreate()

print("Sessão Spark iniciada para Carga GOLD.")

# --- 2. Variáveis de Caminho ---
CAMINHO_SILVER_SUICIDIO = "hdfs://namenode:9000/datalake/silver/suicidio"
CAMINHO_SILVER_DEPRESSAO = "hdfs://namenode:9000/datalake/silver/depressao"
CAMINHO_GOLD = "hdfs://namenode:9000/datalake/gold/suicidio_depressao"

# Variáveis do Banco de Dados
URL_POSTGRES = "jdbc:postgresql://postgres:5432/superset"
TABELA_POSTGRES = "fato_suicidio_depressao"
PROPS_POSTGRES = {
    "user": "superset",
    "password": "superset",
    "driver": "org.postgresql.Driver"
}

# --- 3. Leitura da Camada SILVER ---
print("Lendo DataFrames da Camada SILVER no HDFS...")
df_suicidio = spark.read.parquet(CAMINHO_SILVER_SUICIDIO)
df_depressao = spark.read.parquet(CAMINHO_SILVER_DEPRESSAO)

# Renomeando colunas para o Join
df_suicidio = df_suicidio.withColumnRenamed("Valor", "Taxa_Suicidio")
df_depressao = df_depressao.withColumnRenamed("Valor", "Taxa_Depressao")

# --- 4. JOIN (Modelagem da Camada GOLD) ---
df_gold = df_suicidio.alias("s") \
    .join(
        df_depressao.alias("d"),
        [
            col("s.Pais") == col("d.Pais"),
            col("s.Ano") == col("d.Ano"),
            col("s.Sexo") == col("d.Sexo")
        ],
        "inner"
    ) \
    .select(
        col("s.Pais"),
        col("s.Ano"),
        col("s.Sexo"),
        col("s.Taxa_Suicidio"),
        col("d.Taxa_Depressao")
    )

print("Resultado da Camada GOLD (Join):")
df_gold.show(10, truncate=False)

# --- 5. Carga da Camada GOLD (HDFS e PostgreSQL) ---

# Salvando no HDFS em Parquet
print(f"Salvando arquivo GOLD em Parquet no HDFS em: {CAMINHO_GOLD}")
df_gold.write.mode("overwrite").parquet(CAMINHO_GOLD)

# Salvando no PostgreSQL (para consumo de BI)
print(f"Salvando tabela no PostgreSQL: {TABELA_POSTGRES}")
df_gold.write \
    .mode("overwrite") \
    .jdbc(url=URL_POSTGRES, table=TABELA_POSTGRES, properties=PROPS_POSTGRES)

print("Carga GOLD finalizada com sucesso!")

# --- 6. Finalização ---
spark.stop()
print("Sessão Spark de Carga GOLD finalizada.")